In [1]:
import argparse

import numpy as np
import pandas as pd
import scipy.sparse as sp
import torch
from sklearn.model_selection import KFold
from tqdm import tqdm

In [2]:
%load_ext autoreload
%autoreload 2

from load_data import load_data
from model import GModel
from myutils import roc_auc, translate_result
from optimizer import Optimizer
from sampler import NewSampler

In [3]:
class Args:
    def __init__(self):
        self.device = "cpu"  # cuda:number or cpu
        self.data = "gdsc1"  # Dataset{gdsc or ccle}


args = Args()
res, drug_finger, exprs, mut, cna, null_mask, pos_num = load_data(args)

load gdsc1


In [4]:
cell_sum = np.sum(res, axis=1)
drug_sum = np.sum(res, axis=0)

target_dim = [
    0,  # Cell
    # 1  # Drug
]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
def mofgcn_new(
    exprs,
    cna,
    mut,
    drug_finger,
    res_mat,
    null_mask,
    target_dim,
    target_index,
    evaluate_fun,
    args,
    seed,
    device,
):
    sampler = NewSampler(res_mat, null_mask, target_dim, target_index, seed)
    val_labels = sampler.test_data[sampler.test_mask] 

    if len(np.unique(val_labels)) < 2:
        print(f"Target {target_index} skipped: Validation set has only one class.")
        return None, None 

    model = GModel(
        adj_mat=sampler.train_data,
        gene=exprs,
        cna=cna,
        mutation=mut,
        sigma=2,
        k=11,
        iterates=3,
        feature_drug=drug_finger,
        n_hid1=192,
        n_hid2=36,
        alpha=5.74,
        device=device,
    )
    opt = Optimizer(
        model,
        sampler.train_data,
        sampler.test_data,
        sampler.test_mask,
        sampler.train_mask,
        roc_auc,
        lr=5e-4,
        epochs=1000,
        device=device,
    ).to(device)
    epoch, true_data, predict_data = opt()
    return true_data, predict_data

In [6]:
n_kfold = 1
true_data_s = pd.DataFrame()
predict_data_s = pd.DataFrame()
for dim in target_dim:
    for seed, target_index in enumerate(tqdm(np.arange(res.shape[dim]))):
        if dim:
            if drug_sum[target_index] < 10:
                continue
        else:
            if cell_sum[target_index] < 10:
                continue
        epochs = []
        for fold in range(n_kfold):
            true_data, predict_data = mofgcn_new(
                exprs=exprs,
                cna=cna,
                mut=mut,
                drug_finger=drug_finger,
                res_mat=res,
                null_mask=null_mask,
                target_dim=dim,
                target_index=target_index,
                evaluate_fun=roc_auc,
                args=args,
                seed=seed,
                device=device,
            )

        true_data_s = pd.concat(
            [true_data_s, translate_result(true_data)], ignore_index=True
        )
        predict_data_s = pd.concat(
            [predict_data_s, translate_result(predict_data)], ignore_index=True
        )

  0%|          | 0/916 [00:00<?, ?it/s]

epoch:   0 loss:0.666842 auc:0.7328
epoch:  20 loss:0.326294 auc:0.9619
epoch:  40 loss:0.245101 auc:0.9731
epoch:  60 loss:0.218875 auc:0.9769
epoch:  80 loss:0.209594 auc:0.9769
epoch: 100 loss:0.203713 auc:0.9773
epoch: 120 loss:0.197820 auc:0.9785
epoch: 140 loss:0.194403 auc:0.9789
epoch: 160 loss:0.192896 auc:0.9789
epoch: 180 loss:0.191845 auc:0.9781
epoch: 200 loss:0.191864 auc:0.9777
epoch: 220 loss:0.190489 auc:0.9777


  0%|          | 1/916 [00:11<2:55:53, 11.53s/it]

epoch: 240 loss:0.189674 auc:0.9762
Fit finished.
epoch:   0 loss:1.254310 auc:0.6774
epoch:  20 loss:0.320894 auc:0.9769
epoch:  40 loss:0.246896 auc:0.9798
epoch:  60 loss:0.220178 auc:0.9772
epoch:  80 loss:0.210168 auc:0.9792
epoch: 100 loss:0.203317 auc:0.9785
epoch: 120 loss:0.196363 auc:0.9785
epoch: 140 loss:0.193730 auc:0.9785
epoch: 160 loss:0.192339 auc:0.9785
epoch: 180 loss:0.191549 auc:0.9779


  0%|          | 2/916 [00:19<2:24:21,  9.48s/it]

...

epoch: 180 loss:0.191905 auc:0.9513
Fit finished.
epoch:   0 loss:0.823964 auc:0.4003
epoch:  20 loss:0.325629 auc:0.9535
epoch:  40 loss:0.246817 auc:0.9695
epoch:  60 loss:0.218278 auc:0.9726
epoch:  80 loss:0.209516 auc:0.9752
epoch: 100 loss:0.203792 auc:0.9752
epoch: 120 loss:0.198299 auc:0.9747
epoch: 140 loss:0.194811 auc:0.9752
epoch: 160 loss:0.193174 auc:0.9752
epoch: 180 loss:0.192053 auc:0.9757
epoch: 200 loss:0.191365 auc:0.9762
epoch: 220 loss:0.190895 auc:0.9773
epoch: 240 loss:0.190042 auc:0.9773
epoch: 260 loss:0.195494 auc:0.9773
epoch: 280 loss:0.189305 auc:0.9778
epoch: 300 loss:0.188170 auc:0.9783
epoch: 320 loss:0.187807 auc:0.9799
epoch: 340 loss:0.188528 auc:0.9793
epoch: 360 loss:0.186029 auc:0.9788
epoch: 380 loss:0.184675 auc:0.9783


100%|█████████▉| 915/916 [7:17:41<00:04,  4.02s/it]

epoch: 400 loss:0.183498 auc:0.9768
Fit finished.
epoch:   0 loss:0.831987 auc:0.5194
epoch:  20 loss:0.298552 auc:0.9882
epoch:  40 loss:0.228308 auc:0.9921
epoch:  60 loss:0.213940 auc:0.9941
epoch:  80 loss:0.207774 auc:0.9941
epoch: 100 loss:0.202561 auc:0.9941
epoch: 120 loss:0.196915 auc:0.9941
epoch: 140 loss:0.194010 auc:0.9941
epoch: 160 loss:0.192519 auc:0.9941
epoch: 180 loss:0.191389 auc:0.9941
epoch: 200 loss:0.190976 auc:0.9947
epoch: 220 loss:0.189884 auc:0.9947
epoch: 240 loss:0.188893 auc:0.9961
epoch: 260 loss:0.188282 auc:0.9961
epoch: 280 loss:0.187680 auc:0.9961
epoch: 300 loss:0.185878 auc:0.9961
epoch: 320 loss:0.184543 auc:0.9941


100%|██████████| 916/916 [7:17:45<00:00, 28.67s/it]

epoch: 340 loss:0.182743 auc:0.9915
Fit finished.


In [7]:
true_data_s.to_csv(f"new_cell_true_{args.data}.csv")
predict_data_s.to_csv(f"new_cell_pred_{args.data}.csv")